In [ ]:
# this script divides the input image slices into 256 * 256 image patches, 
# perform data augmentation, and generate the training and validation sets (2 csv files containing paths to image patches). 

In [ ]:
import cv2
import numpy as np
from glob import glob
import csv
import matplotlib.pyplot as plt

In [ ]:
slice_number = 'Slice_1' # change here please!!!
mask = cv2.imread(slice_number + '_weed_mask.png')[:,:,0]
weed = cv2.imread(slice_number + '.png')
height, width = mask.shape
patch_size = 256
grid_size_x = int(height/patch_size) - 1
grid_size_y = int(width/patch_size ) - 1

In [ ]:
n = 0
for i in range(grid_size_x):
    for j in range(grid_size_y):
        patch_image = weed[i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size, :]
        patch_mask = mask[i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size]
        
        if np.mean(patch_mask) > 0:
            n = n + 1
        
            output_image_name = './preprocessing/' + slice_number + '_image_patch_' + str(i) + '_' + str(j) + '.png'
            output_mask_name = './preprocessing/' + slice_number + '_mask_patch_' + str(i) +  '_' +  str(j) + '.png'

            cv2.imwrite(output_image_name, patch_image)
            cv2.imwrite(output_mask_name, patch_mask)
            
print(n)

In [ ]:
image_names = glob('./preprocessing/*image_patch*')

In [ ]:
### Data augmentations

for i in range(len(image_names)):
    image = cv2.imread(image_names[i])
    mask = cv2.imread(image_names[i].replace('image', 'mask'))
    
    image_flipped = cv2.flip(image,0)
    mask_flipped = cv2.flip(mask,0)
    
    cv2.imwrite(image_names[i].replace('image_patch', 'image_patch_vertically_flipped'), image_flipped)
    cv2.imwrite(image_names[i].replace('image_patch', 'mask_patch_vertically_flipped'), mask_flipped)
    
    image_flipped = cv2.flip(image,1)
    mask_flipped = cv2.flip(mask,1)
    
    cv2.imwrite(image_names[i].replace('image_patch', 'image_patch_horizontally_flipped'), image_flipped)
    cv2.imwrite(image_names[i].replace('image_patch', 'mask_patch_horizontally_flipped'), mask_flipped)
    

In [ ]:
# repeat the above steps for all input image slices, you can also write a for loop

In [ ]:
### list names of all image patches
image_list = glob('./preprocessing/Slice*image_patch*')

In [ ]:
output_path = './training_data/train.csv'

In [ ]:
with open(output_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['image'])
    for string in image_list:
        csv_writer.writerow([string])

In [ ]:
# randomly move 20% of the data from ./training_data/train.csv to ./training_data/val.csv